In [1]:
import cv2
import torch
from ultralytics import YOLO

In [2]:
def natural_sort_key(filename):
    """Extract numerical values from filenames for correct sorting."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

In [3]:
def detect_objects(model, image_path):
    image = cv2.imread(image_path)
    results = model(image)  # Perform object detection
    return results, image


In [4]:
def get_ceiling_floor_coordinates(results, floor_model):
    ceiling_y = None  # Initialize
    floor_y = None  # Initialize
    best_ceiling = None
    best_floor = None

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())  # Bounding box coords
            confidence = float(box.conf[0].cpu().numpy())  # Confidence score
            class_id = int(box.cls[0].cpu().numpy())  # Class ID
            
            class_name = floor_model.names.get(class_id, "Unknown")

            if class_name == "ceiling":
                if best_ceiling is None or confidence > best_ceiling[1]:
                    best_ceiling = ((x1, y1, x2, y2), confidence)

            if class_name == "floor":
                if best_floor is None or confidence > best_floor[1]:
                    best_floor = ((x1, y1, x2, y2), confidence)

    ceiling_mid, floor_mid = None, None  

    if best_ceiling:
        (_, ceiling_y1, _, ceiling_y2), _ = best_ceiling
        ceiling_y = (ceiling_y1 + ceiling_y2)// 2  # Midpoint

    if best_floor:
        (_, floor_y1, _, floor_y2), _ = best_floor
        floor_y = (floor_y1 + floor_y2)// 2  # Midpoint

    return ceiling_y, floor_y


In [5]:
def detect_height_bottle(image_path, model):
    """Detect Coca-Cola bottles and return their bounding box height in pixels."""
    image = cv2.imread(image_path)
    results = model(image)
    coke_boxes = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates
            height_pixels = y2 - y1  # Bottle height in pixels
            real_height_cm = 26  # Known height of Coca-Cola bottle
            coke_boxes.append((x1, y1, x2, y2))
            pixel_to_cm_ratio = real_height_cm / height_pixels
            estimated_real_height = height_pixels * pixel_to_cm_ratio
            
            return height_pixels, estimated_real_height
    

    return None, None  # If no bottle is detected

In [6]:
def get_floor_coordinates(results):
    """Extract the top Y-coordinate and bounding box of the floor"""
    floor_y_top = None
    floor_box = None
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 1:  # Assuming class 1 is "floor"
                floor_y_top = y_min  # Top Y-coordinate of floor
                floor_box = (x_min, y_min, x_max, y_max)
    
    return floor_y_top, floor_box

In [7]:
def get_propeller_coordinates(results):
    """Extract the bottom Y-coordinates and bounding boxes of propellers"""
    propeller_boxes = []  # Store bounding boxes of detected propellers
    propeller_y_bottoms = []  # Store Y-bottom of propellers
    
    for result in results:
        for box in result.boxes:
            class_id = int(box.cls.cpu().numpy().item())  # Get class ID
            x_min, y_min, x_max, y_max = map(int, box.xyxy[0])  # Bounding box
            
            if class_id == 0:  # Assuming class 0 is "propeller"
                propeller_y_bottoms.append(y_max)  # Store bottom Y-coordinate
                propeller_boxes.append((x_min, y_min, x_max, y_max))
    
    return propeller_y_bottoms, propeller_boxes

In [8]:
def detect_height_bottle(image_path, model):
    """Detect Coca-Cola bottles and return their bounding box height in pixels."""
    image = cv2.imread(image_path)
    results = model(image)
    coke_boxes = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates
            height_pixels = y2 - y1  # Bottle height in pixels
            real_height_cm = 26  # Known height of Coca-Cola bottle
            coke_boxes.append((x1, y1, x2, y2))
            pixel_to_cm_ratio = real_height_cm / height_pixels
            estimated_real_height = height_pixels * pixel_to_cm_ratio
            
            return height_pixels, estimated_real_height
    

    return None, None  # If no bottle is detected

In [9]:
def calculate_height(ceiling_y, floor_y, calibration_obj):
    """Calculate ceiling height using a calibration object (Coca-Cola bottle)."""
    if ceiling_y is None or floor_y is None:
        print("Error: Ceiling or Floor not detected!")
        return None  # Avoid TypeError

    pixel_height = floor_y - ceiling_y

    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        pixel_to_cm_ratio = real_height / obj_pixel_height
        return pixel_height * pixel_to_cm_ratio
    
    return pixel_height  # Return pixel height if no calibration object


In [10]:
def draw_detections(image, propeller_boxes, floor_box, calibration_obj, floor_results):
    best_ceiling = None
    best_floor = None  # Ensure variables are initialized

    for i, (x1, y1, x2, y2) in enumerate(propeller_boxes):  # Unpacking tuple directly
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])  # Ensure integer values
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Calculate height from floor to propeller
        pixel_height = y1 - floor_box[1]
        real_height = calculate_height(y1, floor_box[1], calibration_obj)
        height_text = f"Height: {real_height:.2f} cm"
        
        # Draw text label
        cv2.putText(image, height_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Label the calibration object
    if calibration_obj:
        real_height, obj_pixel_height = calibration_obj
        cal_text = f"Calibration: {real_height:.2f} cm ({obj_pixel_height} px)"
        cv2.putText(image, cal_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    
    # Process results
    for result in floor_results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())  # Bounding box coordinates
            confidence = float(box.conf[0].cpu().numpy())  # Confidence score
            class_id = int(box.cls[0].cpu().numpy())  # Class ID

            # Get class name correctly
            class_name = floor_model.names.get(class_id, "Unknown")  # Ensure we get the correct class label

            # Store the most confident "ceiling" and "floor" detection
            if class_name == "ceiling":
                if best_ceiling is None or confidence > best_ceiling[1]:
                    best_ceiling = ((x1, y1, x2, y2), confidence, class_name)

            if class_name == "floor":
                if best_floor is None or confidence > best_floor[1]:
                    best_floor = ((x1, y1, x2, y2), confidence, class_name)    # f.write("\nDetection Results (Ceiling Height and AC Height):\n")
    # for result in image_results:
    #     f.write(result + "\n")

    # Draw only the best detections on the image
    for best in [best_ceiling, best_floor]:
        if best is not None:  # Ensure best_ceiling and best_floor are not None
            (x1, y1, x2, y2), confidence, class_name = best
            label = f'{class_name}: {confidence:.2f}'
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return image


In [11]:
def draw_annotations(image, coke_boxes, ceiling_y, floor_y, pixel_to_cm_ratio=None):
    """Draws bounding boxes, vertical line, and labels on the image."""
    annotated_image = image.copy()
    
    # Draw Coca-Cola bounding boxes
    for (x1, y1, x2, y2, height_pixels) in coke_boxes:
        cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
        cv2.putText(annotated_image, f"{height_pixels}px", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Draw vertical line from floor to ceiling
    if ceiling_y is not None and floor_y is not None:
        mid_x = image.shape[1] // 2  # Center x-coordinate
        cv2.line(annotated_image, (mid_x, ceiling_y), (mid_x, floor_y), (0, 0, 255), 2)  # Red line
        
        # Calculate real-world ceiling height if calibration object exists
        pixel_height = floor_y - ceiling_y
        if pixel_to_cm_ratio:
            real_ceiling_height = pixel_height * pixel_to_cm_ratio
            height_text = f"Height: {real_ceiling_height:.2f} cm"
        else:
            height_text = f"Height: {pixel_height} pixels"

        # Add text label
        cv2.putText(annotated_image, height_text, (mid_x + 10, (ceiling_y + floor_y) // 2),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    return annotated_image

In [16]:
import os
import re
import cv2
import numpy as np
from ultralytics import YOLO

def natural_sort_key(filename):
    """Extract numerical values from filenames for correct sorting."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', filename)]

# Define folders
image_folder = "Input Dataset folder"
output_folder = "Output_test_folder_combined2"
propeller_output_folder = "Output_test_folder_propeller_only1"
os.makedirs(output_folder, exist_ok=True)

# Load models
coke_model = YOLO("best (8).pt")          # For calibration (coca bottle)
box_propeller_model = YOLO("best (20).pt")  # For box propellers
round_propeller_model = YOLO("best (21).pt")  # For round propellers
floor_model = YOLO("best (5).pt")           # For floor detection

def calculate_pixel_to_cm_ratio(estimated_real_height, height_pixels):
    """Calculate pixel-to-cm ratio using the detected Coca-Cola bottle height."""
    return estimated_real_height / height_pixels if height_pixels else None

def detect_objects(model, image_path):
    """Run object detection using a given YOLO model."""
    image = cv2.imread(image_path)
    results = model(image)
    return results, image

def detect_and_measure_propellers(image_path, box_model, round_model, pixel_to_cm_ratio):
    """
    Detect and measure both round and box propellers in the image.
    Returns:
       measurements: list of tuples.
         For box: ("Box", width_cm, height_cm, vertical_height)
         For round: ("Round", diameter_cm, circumference_cm, vertical_height)
       image: the annotated image.
    Note: This function assumes that global variables `floor_y` and `calibration_obj` are set.
    """
    box_results, image = detect_objects(box_model, image_path)
    round_results, _ = detect_objects(round_model, image_path)
    
    measurements = []
    CONFIDENCE_THRESHOLD_BOX = 0.65
    CONFIDENCE_THRESHOLD_ROUND = 0.65

    # Process box propellers
    for result in box_results:
        for i, box in enumerate(result.boxes.xyxy):
            confidence = float(result.boxes.conf[i])
            if confidence < CONFIDENCE_THRESHOLD_BOX:
                continue
            x1, y1, x2, y2 = map(int, box.tolist())
            width_px = x2 - x1
            height_px = y2 - y1
            width_cm = width_px * pixel_to_cm_ratio if pixel_to_cm_ratio else 0
            height_cm = height_px * pixel_to_cm_ratio if pixel_to_cm_ratio else 0
            # Calculate vertical height from the bottom of the box (y2) to the floor
            propeller_height = calculate_height(y2, floor_y, calibration_obj) if calibration_obj else 0
            measurements.append(("Box", width_cm, height_cm, propeller_height))
            
            # Draw box and annotate
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 255), 2)
            label = f"W:{width_cm:.2f} cm, H:{height_cm:.2f} cm" if pixel_to_cm_ratio else "Box: Unknown"
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            x_mid = (x1 + x2) // 2
            cv2.line(image, (x_mid, y2), (x_mid, floor_y), (0, 255, 255), 2)
            text_pos = (x_mid + 10, (y2 + floor_y) // 2)
            label_height = f"H: {propeller_height:.2f} cm" if calibration_obj else "Height: Unknown"
            cv2.putText(image, label_height, text_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    
    # Process round propellers
    for result in round_results:
        for i, box in enumerate(result.boxes.xyxy):
            confidence = float(result.boxes.conf[i])
            if confidence < CONFIDENCE_THRESHOLD_ROUND:
                continue
            x1, y1, x2, y2 = map(int, box.tolist())
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            radius_px = max((x2 - x1), (y2 - y1)) // 2  # approximate radius
            diameter_px = 2 * radius_px
            diameter_cm = diameter_px * pixel_to_cm_ratio if pixel_to_cm_ratio else 0
            circumference_cm = np.pi * diameter_cm if pixel_to_cm_ratio else 0
            # Calculate vertical height from the bottom of the circle to the floor
            bottom_y = center_y + radius_px
            propeller_height = calculate_height(bottom_y, floor_y, calibration_obj) if calibration_obj else 0
            measurements.append(("Round", diameter_cm, circumference_cm, propeller_height))
            
            # Draw circle and annotate
            cv2.circle(image, (center_x, center_y), radius_px, (0, 0, 255), 2)
            cv2.line(image, (center_x, bottom_y), (center_x, floor_y), (0, 0, 255), 2)
            text_pos = (center_x + 10, (bottom_y + floor_y) // 2)
            label_height = f"H: {propeller_height:.2f} cm" if calibration_obj else "Height: Unknown"
            cv2.putText(image, label_height, text_pos, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            circ_label = f"C: {circumference_cm:.2f} cm" if pixel_to_cm_ratio else "Circumference: Unknown"
            cv2.putText(image, circ_label, (x1, y1 - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    return measurements, image

# Dictionaries to store results (if needed)
ceil_heights = {}
Propeller_heights = {}
image_results = []

# Get and sort image filenames
image_files = sorted(
    [f for f in os.listdir(image_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))],
    key=natural_sort_key
)
j=0
for image_name in image_files:
    image_path = os.path.join(image_folder, image_name)
    # Detect Coca-Cola bottle for calibration.
    height_pixels, estimated_real_height = detect_height_bottle(image_path, coke_model)
    calibration_obj = (estimated_real_height, height_pixels) if height_pixels else None
    # Get floor (and ceiling) coordinates.
    floor_results, _ = detect_objects(floor_model, image_path)
    # (Assume get_ceiling_floor_coordinates returns (ceiling_y, floor_y))
    ceiling_y, floor_y = get_ceiling_floor_coordinates(floor_results, floor_model)
    if ceiling_y is None or floor_y is None:
        ceiling_height = "XXXXX"
        formatted_ceil_height = "XXXXX"
    elif calibration_obj:
        ceiling_height = calculate_height(ceiling_y, floor_y, calibration_obj)
    else:
        ceiling_height = "XXXXX"
        formatted_ceil_height = "XXXXX"
    print(f"Ceiling Height for {image_name}: {ceiling_height}")
    if ceiling_height != "XXXXX":
        if isinstance(ceiling_height, torch.Tensor):
            formatted_ceil_height = round(ceiling_height.item(), 2)  # Extract scalar if it's a tensor
        else:
            formatted_ceil_height = round(float(ceiling_height), 2)  # Convert string/float to rounded float
        ceil_heights[image_name] = ceiling_height
    pixel_to_cm_ratio = calculate_pixel_to_cm_ratio(estimated_real_height, height_pixels)
    # Detect and measure propellers.
    # Note: The function uses the global variables floor_y and calibration_obj.
    measurements, processed_image = detect_and_measure_propellers(image_path, box_propeller_model, round_propeller_model, pixel_to_cm_ratio)
    
    # Draw vertical reference line for ceiling height.
    if ceiling_y is not None and floor_y is not None and ceiling_height != "XXXXX":
        mid_x = processed_image.shape[1] // 2
        cv2.line(processed_image, (mid_x, ceiling_y), (mid_x, floor_y), (0, 255, 0), 3)
        text_position = (mid_x + 10, (ceiling_y + floor_y) // 2)
        cv2.putText(processed_image, f"{ceiling_height:.2f} cm", text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    coca_bottle_detected = 1 if calibration_obj else 0
    round_circumference = 0
    round_prop_height = 0
    box_length = 0
    box_width = 0
    box_prop_height = 0
    for meas in measurements:
        if meas[0] == "Round":
            round_circumference = int(meas[2])
            round_prop_height = int(meas[3]) 
        elif meas[0] == "Box":
            box_length = int(meas[1])
            box_width = int(meas[2])
            box_prop_height = int(meas[3])
    base_name = image_name.split('.')[0]
    if coca_bottle_detected:
        if round_circumference > 0:
            new_filename = f"{base_name}-{coca_bottle_detected}-{round_circumference}-{round_circumference}-{round_prop_height}.jpg"
        elif box_length > 0:
            new_filename = f"{base_name}-{coca_bottle_detected}-{box_length}-{box_width}-{box_prop_height}.jpg"
        else:
            new_filename = f"{base_name}-{coca_bottle_detected}-0-0-0.jpg"
    else:
        new_filename = f"{base_name}-2-0-0-0.jpg"
    output_path = os.path.join(output_folder, new_filename)
    cv2.imwrite(output_path, processed_image)
    print(f"Processed Image saved at: {output_path}")
    propeller_heights_details = []
    for idx, meas in enumerate(measurements):
        if meas[0] == "Box":
            print(f"Box Propeller {idx + 1} -> Length: {meas[1]} cm, Width: {meas[2]} cm, Height: {meas[3]} cm")
        else:
            print(f"Round Propeller {idx + 1} -> Diameter: {meas[1]} cm, Circumference: {meas[2]} cm, Height: {meas[3]} cm")   
        propeller_heights_details.append(f"P{idx + 1}: {meas[3]:.2f} cm")
    ac_height = ", ".join(propeller_heights_details) if propeller_heights_details else "YYYY"
    j = j+1
    image_results.append(f"Photo {j} | ID {new_filename} | Ceiling height {formatted_ceil_height}cm | AC Height {ac_height}")
# Print final results
print("\nFinal Results:")
for result in image_results:
    print(result)

results_txt = os.path.join(output_folder, "detect_results.txt")
# Write the results into a text file with separate sections.
with open(results_txt, "w") as f:
    f.write("\nDetection Results (Ceiling Height and AC Height):\n")
    for result in image_results:
        f.write(result + "\n")

print(f"Results saved to {results_txt}")
# print(f"\nDetection results saved at: {txt_output_path}")

/home/priyansh/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


0: 800x608 1 750ml coke, 231.6ms
Speed: 4.1ms preprocess, 231.6ms inference, 5.5ms postprocess per image at shape (1, 3, 800, 608)

0: 640x480 2 ceilings, 2 floors, 4 wallss, 205.9ms
Speed: 1.5ms preprocess, 205.9ms inference, 7.9ms postprocess per image at shape (1, 3, 640, 480)
Ceiling Height for imageA.jpeg: 307.6259460449219

0: 800x608 (no detections), 195.4ms
Speed: 3.2ms preprocess, 195.4ms inference, 0.3ms postprocess per image at shape (1, 3, 800, 608)

0: 800x608 (no detections), 196.3ms
Speed: 2.5ms preprocess, 196.3ms inference, 0.3ms postprocess per image at shape (1, 3, 800, 608)
Processed Image saved at: Output_test_folder_combined/imageA-1-0-0-0.jpg

0: 608x800 (no detections), 179.2ms
Speed: 3.0ms preprocess, 179.2ms inference, 0.4ms postprocess per image at shape (1, 3, 608, 800)

0: 480x640 1 ceiling, 1 floor, 2 wallss, 192.7ms
Speed: 1.2ms preprocess, 192.7ms inference, 2.3ms postprocess per image at shape (1, 3, 480, 640)
Ceiling Height for imagea.jpg: XXXXX

0: 60

In [20]:
import os

output_folder = "Output_test_folder_combined"
results_txt = os.path.join(output_folder, "results.txt")

# Prepare groups
only_coke = []    # Only Coca-Cola bottle detection: coca_flag == "1" and measurements are 0-0-0
round_prop = []   # Coca-Cola bottle + round propeller: coca_flag == "1" and m1==m2 != 0
box_prop = []     # Coca-Cola bottle + box propeller: coca_flag == "1" and m1 != m2
unknown = []      # Unknown detection: coca_flag == "2"

# Iterate over files in the output folder
for filename in os.listdir(output_folder):
    if not filename.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    base = os.path.splitext(filename)[0]
    parts = base.split('-')
    
    # Expecting format: <imageName>-<coca_flag>-<m1>-<m2>-<m3>
    if len(parts) < 5:
        continue

    image_id = parts[0]      # e.g., imageA
    coca_flag = parts[1]     # "1" for detected coke, "2" for unknown
    try:
        m1 = int(parts[2])
        m2 = int(parts[3])
        m3 = int(parts[4])
    except ValueError:
        continue

    # Group the file based on the coca_flag and measurement values.
    if coca_flag == "1":
        if m1 == 0 and m2 == 0 and m3 == 0:
            only_coke.append(f"{image_id} {coca_flag}-{m1}-{m2}-{m3}")
        elif m1 == m2 and m1 != 0:
            round_prop.append(f"{image_id} {coca_flag}-{m1}-{m2}-{m3}")
        elif m1 != m2:
            box_prop.append(f"{image_id} {coca_flag}-{m1}-{m2}-{m3}")
    elif coca_flag == "2":
        unknown.append(f"{image_id} {coca_flag}-{m1}-{m2}-{m3}")

# Write the results into a text file with separate sections.
with open(results_txt, "w") as f:
    f.write("For only Coca-Cola bottle detection:\n")
    for line in only_coke:
        f.write(line + "\n")
    
    f.write("\nFor Coca-Cola bottle + round propeller:\n")
    for line in round_prop:
        f.write(line + "\n")
    
    f.write("\nFor Coca-Cola bottle + box propeller:\n")
    for line in box_prop:
        f.write(line + "\n")

    f.write("\nFor unknown detection (no valid coke bottle):\n")
    for line in unknown:
        f.write(line + "\n")

print(f"Results saved to {results_txt}")


Results saved to Output_test_folder_combined/results.txt
